# Client Churn Prediction
### CRISP-DM Cycle 4
---
The Top Bank company operates in Europe with a bank account as the main product, this product can keep client's salary and make payments. This account doesn't have any cost in the first 12 months, however, after that time trial, the client needs to rehire the bank for upcoming 12 months and redo this process every year. Recently the Analytics Team noticed that the churn rate is increasing.

As a Data Science Consultant, you need to create an action plan to decrease the number of churn customers and show the financial return on your solution.
At the end of your consultancy, you need to deliver to the TopBottom CEO a model in production, which will receive a customer base via API and return that same base with an extra column with the probability of each customer going into churn.
In addition, you will need to provide a report reporting your model's performance and the financial impact of your solution. Questions that the CEO and the Analytics team would like to see in their report:

1.  What is Top Bank's current Churn rate?
2.  How does the churn rate vary monthly?
3.  What is the performance of the model in classifying customers as churns
4.  What is the expected return, in terms of revenue, if the company uses its model to avoid churn from customers?

> Disclaimer: This is a fictional business case

## 0. Preparation

### 0.1 Imports & Settings

In [20]:
from IPython.core.display import HTML
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from src.utils_feature_selection import multiple_histplots, apply_log_transformation, apply_one_hot_encoder, apply_standard_scaler, apply_min_max_scaler, apply_robust_scaler, plot_feature_importance, select_features_with_rfe

In [21]:
def jupyter_settings():
    """
    Plots pre settings.
    """

    %matplotlib inline
    plt.style.use("seaborn-v0_8-whitegrid")
    plt.rcParams["figure.figsize"] = [25, 12]
    plt.rcParams["font.size"] = 24
    display(HTML("<style>.container {width:100% !important;}</style>"))
    sns.set()


jupyter_settings()

seed = 42

### 0.3 Path

In [22]:
# found the main project folders
path = Path().resolve().parent
data_path = path / "data/processed"
fig_path = path / "pics"


### 0.4 Data

This dataset is available [here](https://www.kaggle.com/mervetorkan/churndataset).


**Data fields**

- **RowNumber**: the number of the columns
- **CustomerID**: unique identifier of clients
- **Surname**: client's last name
- **CreditScore**: clients credit score for the financial market
- **Geography**: the country of the client
- **Gender**: the gender of the client
- **Age**: the client's age
- **Tenure**: number of years the client is in the bank 
- **Balance**: the amount that the client has in their account 
- **NumOfProducts**: the number of products that the client bought 
- **HasCrCard**: if the client has a credit card 
- **IsActiveMember**: if the client is active (within the last 12 months) 
- **EstimateSalary**: estimate of annual salary of clients 
- **Exited**: if the client is a churn (*target variable*)

In [23]:
X_train = pd.read_parquet(data_path / "X_train.parquet")
X_test = pd.read_parquet(data_path / "X_test.parquet")
X_val = pd.read_parquet(data_path / "X_val.parquet")
y_train = pd.read_pickle(data_path / "y_train.pkl")
y_test = pd.read_pickle(data_path / "y_test.pkl")
y_val = pd.read_pickle(data_path / "y_val.pkl")

## 7. Bussiness Translation

In [ ]:
X_data = pd.read_pickle(homepath + 'data/processed/X_data.pkl')
y_data = pd.read_pickle(homepath + 'data/processed/y_data.pkl')

X_test = pd.read_pickle(homepath + 'data/processed/X_test.pkl')
y_test = pd.read_pickle(homepath + 'data/processed/y_test.pkl')

estimated_salary = pd.read_pickle(homepath + 'data/processed/estimated_salary.pkl')

model = pickle.load(open(homepath + 'models/model.pkl', 'rb'))
threshold = pickle.load(open(homepath + 'models/threshold.pkl', 'rb'))
yhat_proba = model.predict_proba(X_test)[: , 1]
yhat = (yhat_proba >= threshold).astype(int)

### 7.1 What is Top Bank's current Churn rate?

In [ ]:
churn_rate = df1_cat[df1_cat['exited'] == 'yes']['exited'].count()/(len(df1_cat))
round(churn_rate*100, 2)

20.37

The current churn rate is 20.37%

### 7.2 How does the churn rate vary monthly?

In [ ]:
monthly_churn_rate = churn_rate_per_month(df1)

print(f' The churn rate varies {round(monthly_churn_rate, 2)} per month.')

 The churn rate varies 8.33 per month.


### 7.2 What is the performance of the model in classifying customers as churns?


In [ ]:
metrics_cv([model], X_data, y_data, threshold = 0.43, verbose = False)


Please wait a moment - Doing CV


Finished, check the results


,Model_Name,Threshold,Balanced_Accuracy Mean,Balanced_Accuracy STD,Precision Mean,Precision STD,Recall Mean,Recall STD,F1 Score Mean,F1 Score STD,ROCAUC Mean,ROCAUC STD
0,CatBoostClassifier,0.430,0.761,0.011,0.536,0.005,0.671,0.026,0.596,0.012,0.761,0.761


### 7.3 What is the expected return, in terms of revenue, if the company uses its model to avoid churn from customers?

#### 7.3.1 Creating DataFrame

In [ ]:
#df2['estimated_salary_regular'] = estimated_salary

In [ ]:
# df2 is a dataframe before the transformations
salary_mean = round(df2['estimated_salary'].mean(), 2)

# Predictions and Results
y_test_frame = y_test.to_frame().reset_index(drop = True)
y_proba = pd.DataFrame(yhat_proba).rename(columns = {0: 'probability'}).reset_index(drop = True)
y_predict = pd.DataFrame(yhat).rename(columns = {0: 'prediction'}).reset_index(drop = True)

# Estimated salary without mms
estimated_salary_frame = estimated_salary.to_frame().reset_index(drop = True)

# Creating a dataframe with the results
df_simulation = pd.concat((y_test_frame, y_proba, y_predict, estimated_salary_frame), axis = 1)

# Verify threshold
df_simulation['threshold'] = df_simulation['probability'].apply(lambda x: 'negative' if x <= 0.4 else 'positive')

# Reorder columns
df_simulation = df_simulation[['estimated_salary' , 'exited', 'prediction', 'probability', 'threshold']]

In [ ]:
salary_mean

100090.24

In [ ]:
df_simulation.head()

,estimated_salary,exited,prediction,probability,threshold
0,94283.090,0,0,0.067,negative
1,55072.930,0,0,0.348,negative
2,105433.530,0,0,0.039,negative
3,33950.080,0,0,0.178,negative
4,168544.850,0,0,0.224,negative


#### 7.3.2 Return per Client

Considering a return per client:
- *15%* if the `estimated_salary` is lower than avg;
- *20%* if the `estimated_salary` is equal to the avg and is also less than two times the avg;
- *25%* if the `estimated_salary` is two times higher or more than avg;

In [ ]:
df_simulation['financial_return'] = df_simulation['estimated_salary'].apply(lambda x: x * 0.15 if x < salary_mean
                                                      else x * 0.2 if x >= salary_mean and x < 2 * salary_mean
                                                      else x * 0.25)

return_clients = round(df_simulation['financial_return'].sum(), 2)
print(f'The return of all clients in this dataframe are: ${return_clients}')

The return of all clients in this dataframe are: $38210856.42


In [ ]:
churn_loss = round(df_simulation[df_simulation['exited'] == 1]['financial_return'].sum(), 2)
print(f'The bank is losing ${churn_loss}, that value represents {round ((churn_loss/return_clients)*100, 2)}% of the total return.')

The bank is losing $7517032.21, that value represents 19.67% of the total return.


**Simulation 1 - Discount Coupon of $100**

With a budget of $10000.00, the bank can deliver $100 discount coupons to the top 100 clients with highest probability of churn.

In [ ]:
simulation_1 = top_clients('Simulation 1', df_simulation, 'probability','prediction', 'financial_return', churn_loss, 100, 100)

**Simulation 2 - Discount Cupom of $50**

With a budget of $10000.00, the bank can deliver $50 discount coupons to the top 200 clients with highest probability of churn.

In [ ]:
simulation_2 = top_clients('Simulation 2', df_simulation, 'probability','prediction', 'financial_return', churn_loss, 200, 50)

**Simulation 3 - knapsack with a incentive of $100**

Select the optimal combination of clients that maximize the total returned value , without exceeding the total constraint.

Using the 0-1 knapsack-problem with probabilities with a budget of $10000.00 and $100 coupons.

In [ ]:
df_simulation_3 = df_simulation[df_simulation['prediction'] == 1]
df_simulation_3['incentive'] = 100

simulation_3 = knapsack_solver('Simulation 3', df_simulation_3, 'prediction', 'financial_return', churn_loss, 10000, 'incentive')

**Simulation 4 - knapsack with a incentive list**

Select the optimal combination of clients that maximize the total returned value , without exceeding the total constraint.

Using the 0-1 knapsack-problem with probabilities with a budget of $10000.00:
- p(churn) >= 0.99: Client that will leave
- 0.95 <= p(churn) < 0.99: Client with a high probability to stay with a $200 coupon;
- 0.90 <= p(churn) < 0.99: Client that might stay with a $100 coupon;
- p(churn) < 0.90: Client that might stay with a $500 coupon.



In [ ]:
df_simulation_4 = df_simulation[df_simulation['prediction'] == 1]
  
# list of available incentives as stated above
# the incentive of 20000 is for clients with p(churn) > 0.99, it will be discarded in the knapsack function
incentives_list = [200, 100, 50]
incentives = []
n = len(df_simulation_4)

# set incentive value according churn predicted probability
for i in range(n):
    entry = df_simulation_4.iloc[i]
    if entry['probability'] >= 0.95 and entry['probability']:
        incentives.append(incentives_list[0])
    elif entry['probability'] >= 0.90 and entry['probability'] < 0.95:
        incentives.append(incentives_list[1])
    else:
        incentives.append(incentives_list[2])

df_simulation_4['incentive'] = incentives

simulation_4 = knapsack_solver('Simulation 4', df_simulation_4, 'prediction', 'financial_return', churn_loss, 10000, 'incentive')


In [ ]:
comparation = pd.concat([simulation_1, simulation_2, simulation_3, simulation_4], axis = 0, ignore_index = True)
comparation

,Scenario,Recovered Revenue,Loss Recovered,Investment,Profit,ROI,Clients Recovered,Churn Reduction
0,Simulation 1,$1685796.24,22.43%,$10000,$1675796.24,16757.96%,91 clients,22.36%
1,Simulation 2,$2906302.0,38.66%,$10000,$2896302.0,28963.02%,158 clients,38.82%
2,Simulation 3,$2179274.99,28.99%,$10000,$2169274.99,21692.75%,60 clients,100.0%
3,Simulation 4,$2733157.31,36.36%,$10000,$2723157.31,27231.57%,87 clients,100.0%


### 7.4 Bussiness Performance Summary
- The Top Bank's **churn rate is 20.37%**
- The monthly churn rate varies, on average, 8.33%
- The model has a accuracy of 76.1% and a recall of 67.1% in classifying clients as churn
- The bank is **losing *$7,517,032.21* in this dataframe because of the churn**
- The return of all clients in this dataframe are: *$38,210,856.42*
- Using the knapsack approach with an incentive list with coupons of $200, $100 and $50 depending of the probability to client's churn can give:
  - Recovered Revenue: *$2733157.31*
  - Churn Loss Recovered: **100%**
  - Investment: *$10000*
  - Profit: *$2,723,157.31*
  - ROI: **21,913.87%**
  - Potential clients recovered with the model: 87 clients 